In [1]:
import warnings
warnings.filterwarnings("ignore")
import torch
import time
from torch.utils.data import DataLoader, Dataset
import random
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
import json
import transformers

In [2]:
# 导入模型
# 设置设备
device = torch.device("cuda")

# 设定模型位置
path = 'Qwen-main/output_qwen/Final/Qwen1.5_y_1'

# 加载模型
tokenizer = AutoTokenizer.from_pretrained(path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(path, trust_remote_code=True, device_map="auto")
print("模型调用成功")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 7/7 [01:42<00:00, 14.67s/it]


模型调用成功


In [3]:
# 读取数据
data_path = "Qwen-main/data/eval.json"

# 读取json文件内部的内容
f = open(data_path)
content = f.read()
before_prompts = json.loads(content)

prompts = []
y = []
for i in range(len(before_prompts)):
    prompt = before_prompts[i]['conversations'][0]['value'] + ':'
    prompts.append(prompt)
    y.append(before_prompts[i]['conversations'][1]['value'])

print(len(y))

785


In [ ]:
torch.cuda.empty_cache()

In [4]:
# 推理结果
outputs = []
time_ = 0
i = 0
print("开始计时")
for prompt in prompts:
    # list_prompt = prompt
    pt_batch = tokenizer(
        prompt,
        padding=True,
        truncation=True,
        return_tensors="pt",
    ).to(device)
    start_time = time.time()
    pt_outputs = model.generate(**pt_batch, 
                            do_sample=False,
                            top_p = 0.5, 
                            temperature = 0.95,
                            max_new_tokens=900
                           )
    end_time = time.time()
    time_ = time_ + (end_time-start_time)
    
    output = tokenizer.batch_decode(pt_outputs)[0].split('。:')[1].split('<|endoftext|>')[0]
    if len(output) <= 5:
        print("本次生成失败")
        output = '本次生成失败'
    # output = tokenizer.batch_decode(pt_outputs)
    outputs.append(output)
    if i%5==0:
        print("推理进度为", (i/len(prompts) * 100), "%")
    
    i = i+1

# 推理结束后释放GPU
torch.cuda.empty_cache()

print("本次推理用时:", time_, "秒")

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


开始计时


2024-04-15 09:37:49.830985: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-15 09:37:51.760698: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-15 09:37:52.759836: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


推理进度为 0.0 %


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


...

推理进度为 2.547770700636943 %


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


推理进度为 3.1847133757961785 %


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


KeyboardInterrupt: 

In [7]:
predict_y[0]

{'y': '原告与被告租赁合同纠纷一案。原告提出诉求：判令被告支付租金、上下车费、违约金。被被告未到庭，未提交书面意见及证据。经查明被告与原告签订的《建筑施工物资租赁合同》合法有效，被告应支付租金。双方在本次起诉前并没有对租金、材料赔偿等进行总结算，故被告应支付所欠费用的90%给原告。被告与原告进行结算后，一直未支付相应费用，已构成违约。但原告请求的违约金过高，本院酌情调整。依照《合同法》第八条、第一百零七条、第一百一十四条第一款、第三款、第二百一十二条、第二百二十六条，《民事诉讼法》第一百四十四条的规定，判决:1、被告向原告支付租金、维修费等费用；2、被告向原告支付违约金；3、驳回原告其他诉讼请求。',
 'pred': '重庆千年建筑设备租赁有限公司与遵义格林建筑工程有限公司租赁合同纠纷一案。重庆千年公司向遵义格林建司提出诉讼请求：遵义格林建司立即支付截止2017年6月6日的租金、上下车费等共计171178元；遵义格林建司支付违约金34000元。遵义格林建司未到庭参加诉讼，也未向本院提交书面意见及证据。本院认为，遵义格林建司与重庆千年公司签订了《建筑施工物资租赁合同》，并加盖双方的印章。该租赁合同系双方真实意思表示，内容不违反法律、行政法规的禁止性规定，应属合法、有效。遵义格林建司使用了重庆千年公司的建筑物资应承担支付租金的合同义务，遵义格林建司一直未支付相应费用，已构成违约。遵义格林建司在2016年12月27日与重庆千年公司进行结算后，一直未支付相应费用，已构成违约。遵义格林建司在本次起诉前并没有对租金、材料赔偿等进行总结算，故遵义格林建司应支付所欠费用的90%给重庆千年公司。遵义格林建司在2016年12月27日与重庆千年公司进行结算后，一直未支付相应费用，已构成违约。遵义格林建司在本次起诉前并没有对租金、材料赔偿等进行总结算，故遵义格林建司应支付所欠费用的90%给重庆千年公司。遵义格林建司一直未支付相应费用，已构成违约。遵义格林建司在本次起诉前并没有对租金、材料赔偿等进行总结算，故遵义格林建司应支付所欠费用的90%给重庆千年公司。遵义格林建司一直未支付相应费用，已构成违约。遵义格林建司在本次起诉前并没有对租金、材料赔偿等进行总结算，故遵义格林建司应支付所欠费用的90%给重庆千年公司。遵义格林建司一直未支付相应费用，已构成违约。遵义格林建司在本次起诉前并没有

In [6]:
## 将推理到的数据和原数据整合
predict_y = []
for i in range(len(outputs)):
    dict_={}
    dict_['y'] = y[i]
    dict_['pred'] = outputs[i]
    predict_y.append(dict_)
    
print("整合完成")

整合完成


In [ ]:
json_file_path = "Qwen-main/data/notishi.json"
json_file = open(json_file_path, mode='w', encoding='utf-8')
json.dump(predict_y, json_file, ensure_ascii=False)